# ライブラリのインポート

In [1]:
import networkx as nx
import numpy as np
import pulp
import plotly.graph_objects as go

# データ生成

In [2]:
def create_data(grid_size: int) -> tuple[nx.Graph, dict[int, tuple[int, int]]]:
    # grid_graphを使ってグラフを生成
    G = nx.grid_2d_graph(grid_size, grid_size)

    # ノードの位置情報を取得
    positions = {i: (pos[1] + 1, grid_size - pos[0]) for i, pos in enumerate(G.nodes())}

    # ノードの番号を0から順に再割り当て
    mapping = {pos: i for i, pos in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping)

    return G, positions

In [3]:
def create_school_route(
    G: nx.Graph,
    grid_size: int,
    commute_weights: dict[int:int],
    school_index: int,
) -> tuple[list[list[int]], list[int]]:
    paths = []
    node_weights = {node: 1 for node in range(grid_size**2)}
    for important_point in commute_weights.keys():
        # 重要な地点から学校までの最短経路を計算
        path = nx.shortest_path(G, source=important_point, target=school_index, weight="weight")
        paths.append(path)

        # 通学路のノードに対して、該当する重要な地点の重みを加算
        for node in path:
            node_weights[node] += commute_weights[important_point]

    return paths, node_weights

# 最適化問題の定式化と求解

In [4]:
def solve_streetlight_problem(
    G: nx.Graph,
    positions: dict[int, tuple[int, int]],
    max_lamps: int,
    node_weights: dict[int, int],  # 通学路に対する人流の重み
    pre_lit_values: dict[int, float] = {},
    verbose: bool = True,
) -> tuple[pulp.LpProblem, list[float], list[float]]:
    num_nodes = len(G.nodes)

    # PuLPの問題設定
    prob = pulp.LpProblem("Street_Light_Placement", pulp.LpMaximize)

    # 変数
    x = pulp.LpVariable.dicts("x", range(num_nodes), cat="Binary")
    y = pulp.LpVariable.dicts("y", range(num_nodes), cat="Continuous")

    # 目的関数: 各ノードの照度に重みを掛けたものの合計を最大化
    prob += pulp.lpSum(node_weights[j] * y[j] for j in range(num_nodes))

    # 制約条件
    # 各地点の照度は、その地点に設置された街灯と隣接するおよび2点先の街灯からの影響度の合計
    for j in range(num_nodes):
        # 隣接ノード
        neighbors = list(G.neighbors(j))
        # 2点先のノード
        second_neighbors = [n for neighbor in neighbors for n in G.neighbors(neighbor)]
        influence_sum = (
            x[j]
            + pulp.lpSum(
                x[neighbor]
                * (
                    1
                    / (2 * np.sum(np.abs(np.array(positions[j]) - np.array(positions[neighbor]))))
                )
                for neighbor in neighbors
            )
            + pulp.lpSum(
                x[second_neighbor]
                * (
                    1
                    / (
                        2
                        * np.sum(
                            np.abs(np.array(positions[j]) - np.array(positions[second_neighbor]))
                        )
                    )
                )
                for second_neighbor in second_neighbors
                if second_neighbor not in neighbors and second_neighbor != j
            )
        )

        # 既に照度を持つ地点の場合、その初期照度を追加
        if j in pre_lit_values:
            influence_sum += pre_lit_values[j]

        prob += y[j] <= influence_sum

    # 街灯数の制約
    prob += pulp.lpSum(x[i] for i in range(num_nodes)) <= max_lamps

    # 照度の最大値
    for j in range(num_nodes):
        prob += y[j] <= 1

    # 求解
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # 結果の取得
    lamp_status = [pulp.value(x[i]) for i in range(num_nodes)]
    point_covered = [pulp.value(y[j]) for j in range(num_nodes)]

    if verbose:
        print(f"最適化結果 (街灯数={max_lamps}): {pulp.LpStatus[prob.status]}")

    return prob, lamp_status, point_covered

# 結果の可視化

In [5]:
def create_2d_plot(
    G: nx.Graph,
    positions: dict[int, tuple[int, int]],
    lamp_status: list[float],
    point_covered: list[float],
    important_points: list[int],
    school_index: int,
    pre_lit_values: dict[int, float] = {},
    paths: list[list[int]] = [],
    node_weights: dict[int, int] = {},
) -> go.Figure:
    # 凡例の表示状態を保持するフラグ
    is_display_legend_placed_lamps = True
    is_display_legend_station = True
    is_display_legend_school = True
    is_display_legend_covered_points = True
    is_display_legend_uncovered_points = True
    is_display_legend_pre_lit_points = True

    fig = go.Figure()

    # 照度によるエッジの影響度を描画
    for edge in G.edges:
        node1, node2 = edge
        pos1 = positions[node1]
        pos2 = positions[node2]

        # エッジの影響度を計算
        distance = np.linalg.norm(np.array(pos1) - np.array(pos2))
        default_influence = 0.5
        influence = default_influence

        # 隣接ノードに対する影響
        if lamp_status[node1] == 1 or lamp_status[node2] == 1:
            influence += np.ones_like(distance) / distance

        # 2点先のノードに対する影響
        second_neighbors1 = set(G.neighbors(node1)).difference([node2])
        second_neighbors2 = set(G.neighbors(node2)).difference([node1])

        for neighbor in second_neighbors1:
            if lamp_status[neighbor] == 1:
                influence += 1 / (
                    2 * np.sum(np.abs((np.array(pos1) - np.array(positions[neighbor]))))
                )

        for neighbor in second_neighbors2:
            if lamp_status[neighbor] == 1:
                influence += 1 / (
                    2 * np.sum(np.abs(np.array(pos2) - np.array(positions[neighbor])))
                )

        # エッジの太さと色を影響度に基づいて設定
        edge_width = influence * 5  # 影響度を拡大して太さに反映
        if influence > default_influence:
            edge_color = (
                f"rgba(255, 0, 0, {min(influence / 3, 0.8)})"  # 影響度に応じて透明度を設定
            )
        else:
            edge_color = "rgba(0, 0, 0, 1.0)"  # デフォルトの色（黒）で表示

        fig.add_trace(
            go.Scatter(
                x=[pos1[0], pos2[0]],
                y=[pos1[1], pos2[1]],
                mode="lines",
                line=dict(color=edge_color, width=edge_width),
                showlegend=False,
            )
        )

        for path in paths:
            for node1, node2 in zip(path[:-1], path[1:]):
                pos1 = positions[node1]
                pos2 = positions[node2]
                weight = min(node_weights.get(node1, 1), node_weights.get(node2, 1)) * 2
                fig.add_trace(
                    go.Scatter(
                        x=[pos1[0], pos2[0]],
                        y=[pos1[1], pos2[1]],
                        mode="lines",
                        line=dict(color="rgba(0, 255, 255, 0.8)", width=weight),
                        showlegend=False,
                    )
                )

    # ノードを描画
    for i, (node, pos) in enumerate(positions.items()):
        lamp = lamp_status[i]
        cover = point_covered[i]

        if node in important_points:
            color = "blue"
            size = 15
            name = "住宅地や駅など"
            showlegend = is_display_legend_station
            is_display_legend_station = False
        elif node == school_index:
            color = "green"
            size = 15
            name = "日立北高校"
            showlegend = is_display_legend_school
            is_display_legend_school = False
        elif node in pre_lit_values:
            color = "purple"  # 既に照度を持つ地点は紫で表示
            size = 20
            name = "既に明るい地点"
            showlegend = is_display_legend_pre_lit_points
            is_display_legend_pre_lit_points = False
        elif lamp == 1:
            color = "red"
            size = 20
            name = "街灯の位置"
            showlegend = is_display_legend_placed_lamps
            is_display_legend_placed_lamps = False
        else:
            # カバーされているかどうかでノードの色とサイズを調整
            if cover > 0:
                color = "yellow"
                size = int(10 + 10 * cover)  # 照度に応じてサイズを拡大
                name = "照らされている地点"
                showlegend = is_display_legend_covered_points
                is_display_legend_covered_points = False
            else:
                color = "black"
                size = 8
                name = "照らされていない地点"
                showlegend = is_display_legend_uncovered_points
                is_display_legend_uncovered_points = False

        fig.add_trace(
            go.Scatter(
                x=[pos[0]],
                y=[pos[1]],
                mode="markers",
                marker=dict(color=color, size=size, line=dict(color="black", width=1)),
                name=name,
                showlegend=showlegend,
            )
        )

    fig.update_layout(
        title="街灯設置の結果",
        xaxis_title="x 座標",
        yaxis_title="y 座標",
        legend_title="凡例",
        width=800,
        height=650,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
    )

    return fig

# 実行

In [15]:
grid_size = 10
G, positions = create_data(grid_size)

commute_weights = {10: 2, 50: 10}
total_weight = sum(commute_weights.values())
commute_weights = {
    point: 1 + (weight / total_weight) * 5 for point, weight in commute_weights.items()
}

important_points = commute_weights.keys()

school_index = 99
paths, node_weights = create_school_route(G, grid_size, commute_weights, school_index)
commute_route_nodes = list(set(node for node in node_weights if node_weights[node] > 1))


pre_lit_points = [22, 48]
pre_lit_values = {}
for point in pre_lit_points:
    pre_lit_values[point] = 0.5

max_lamps = 5

In [16]:
prob, lamp_status, point_covered = solve_streetlight_problem(
    G, positions, max_lamps, node_weights, pre_lit_values
)

最適化結果 (街灯数=5): Optimal
CPU times: user 21.3 ms, sys: 8.51 ms, total: 29.8 ms
Wall time: 80 ms


In [87]:
fig = create_2d_plot(
    G,
    positions,
    lamp_status,
    point_covered,
    important_points,
    school_index=school_index,
    pre_lit_values=pre_lit_values,
    paths=paths,
    node_weights=node_weights,
)
fig.show()